# Tutorial pyOSRD #6
Creating custom agent

In [1]:
from pyosrd.agents.scheduler_agent import SchedulerAgent
import pandas as pd
from dataclasses import dataclass

## Creating a dumb agent
Create an agent that add 60 seconds of delay at the second zone the first train go throught.

In [2]:

class DumbSchedulerAgent(SchedulerAgent):
    @property
    def steps_extra_delays(self) -> pd.DataFrame:
        """As an example, make train1 stop 60s more in second zone"""
        extra_delays = self.ref_schedule.durations * 0.
        extra_delays.iloc[0, 1] = 60.
        return extra_delays


## Test our agent on all scenarii
We can launch our custom agent on all scenarii implemented in the module OSRD.scenarii.

In [3]:
from pyosrd.agents.scheduler_agent import regulate_scenarii_with_agents
from functools import partial
from pyosrd.schedules.weights import all_steps

agent = DumbSchedulerAgent("dumb")


regulate_scenarii_with_agents("all", agent)

[16:05:32,150] [INFO]  [StandaloneSimulationCommand] Loading infra: tmp/infra.json
[16:05:32,229] [INFO]              [FullInfra] parsing infra
[16:05:32,402] [INFO]              [FullInfra] adaptation to kotlin
[16:05:32,461] [INFO]              [FullInfra] loading signals
[16:05:32,477] [INFO]              [FullInfra] building blocks
[16:05:32,493] [INFO]  [StandaloneSimulationCommand] Loading input: tmp/simulation.json
[16:05:32,957] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.0
[16:05:33,185] [DEBUG]             [Allowance] computing range n°1
[16:05:33,191] [DEBUG]             [Allowance]   computing section n°1
[16:05:33,192] [DEBUG]             [Allowance]   target time = 310.84094999999996
[16:05:33,192] [DEBUG]             [Allowance]     starting attempt 1
[16:05:33,196] [DEBUG]             [Allowance]     envelope time 518.958
[16:05:33,196] [DEBUG]             [Allowance]     starting attempt 2
[16:05:33,196] [DEBUG]             [Allow

,dumb
c1_delay,0.0
c1y2_2trains_conflict,0.0
c1y2_2trains_no_conflict,0.0
c2y11s_conflict_20_trains,0.0
c2y1y2y_2_trains_no_conflict,0.0
c2y1y2y_2_trains_reorder,0.0
c2y1y2y_2_trains_same_order,0.0


## Compare two agents

The function regulate_scenarii_with_agents can be launched on multiple agents at the same time. It returns a pandas DataFrame containing the ponderated delays
for all scenarii.

An agent can be compared to another one using those scenarii.

In [4]:

class DumberSchedulerAgent(SchedulerAgent):
    @property
    def steps_extra_delays(self) -> pd.DataFrame:
        """As an example, make train1 stop 120s more in second zone"""
        extra_delays = self.ref_schedule.durations * 0.
        extra_delays.iloc[0, 1] = 120.
        return extra_delays

To speed up the notebook execution the next call is done only on one scenario "c1_delay".

In [5]:

agent_dumber = DumberSchedulerAgent("dumber")
regulate_scenarii_with_agents("c1_delay", [agent, agent_dumber])

[16:06:06,329] [INFO]  [StandaloneSimulationCommand] Loading infra: tmp/infra.json
[16:06:06,430] [INFO]              [FullInfra] parsing infra
[16:06:06,640] [INFO]              [FullInfra] adaptation to kotlin
[16:06:06,718] [INFO]              [FullInfra] loading signals
[16:06:06,738] [INFO]              [FullInfra] building blocks
[16:06:06,756] [INFO]  [StandaloneSimulationCommand] Loading input: tmp/simulation.json
[16:06:07,299] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.14
[16:06:07,573] [DEBUG]             [Allowance] computing range n°1
[16:06:07,584] [DEBUG]             [Allowance]   computing section n°1
[16:06:07,585] [DEBUG]             [Allowance]   target time = 310.84094999999996
[16:06:07,585] [DEBUG]             [Allowance]     starting attempt 1
[16:06:07,592] [DEBUG]             [Allowance]     envelope time 518.958
[16:06:07,592] [DEBUG]             [Allowance]     starting attempt 2
[16:06:07,593] [DEBUG]             [Allo

,dumb,dumber
c1_delay,0.0,0.0
